# Dynamic Programming – Sprint 3

**INTEGRANTES**
- Fernanda Menon RM 554673
- Gabriel Lacerda RM 556714
- Luiza Macena RM 556237
- Roger Cardoso RM 557230

2ESPW

 Este notebook implementa **estruturas de dados e algoritmos clássicos** aplicados ao consumo de insumos hospitalares.

### Objetivos:
- Registrar consumo em ordem cronológica (**Fila / Queue**).
- Consultar em ordem inversa (**Pilha / Stack**).
- Localizar insumos usando busca **Sequencial** e **Binária**.
- Organizar dados por quantidade (**Merge Sort**) e validade (**Quick Sort**).
- Gerar relatório final.

## 📦 Importações e Definições

In [1]:

from collections import deque
from dataclasses import dataclass
from datetime import datetime, timedelta
import random
from typing import List, Tuple, Any


## 📌 Estrutura de Dados - Consumo de Insumos

In [2]:

@dataclass(frozen=True)
class ConsumptionEvent:
    item: str
    date: datetime
    qty: int
    validity_until: datetime


## 🔄 Estruturas - Fila (Queue) e Pilha (Stack)

In [3]:

class Queue:
    def __init__(self):
        self._dq = deque()
    def enqueue(self, value: Any): self._dq.append(value)
    def dequeue(self): return self._dq.popleft() if self._dq else None
    def __len__(self): return len(self._dq)
    def to_list(self): return list(self._dq)

class Stack:
    def __init__(self):
        self._data = []
    def push(self, value: Any): self._data.append(value)
    def pop(self): return self._data.pop() if self._data else None
    def __len__(self): return len(self._data)
    def to_list(self): return list(self._data)


## 🔍 Algoritmos de Busca

In [4]:

def linear_search(events: List[ConsumptionEvent], predicate):
    return [i for i, ev in enumerate(events) if predicate(ev)]

def binary_search(events_sorted: List[ConsumptionEvent], key: Tuple[str, datetime]) -> int:
    lo, hi = 0, len(events_sorted) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        probe = (events_sorted[mid].item, events_sorted[mid].date)
        if probe == key: return mid
        if probe < key: lo = mid + 1
        else: hi = mid - 1
    return -1


## 📊 Algoritmos de Ordenação

In [5]:

def merge_sort(events: List[ConsumptionEvent], key):
    if len(events) <= 1: return events[:]
    mid = len(events) // 2
    left = merge_sort(events[:mid], key)
    right = merge_sort(events[mid:], key)
    return _merge(left, right, key)

def _merge(left, right, key):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if key(left[i]) <= key(right[j]): result.append(left[i]); i += 1
        else: result.append(right[j]); j += 1
    result.extend(left[i:]); result.extend(right[j:])
    return result

def quick_sort(events: List[ConsumptionEvent], key, low=0, high=None):
    if high is None: high = len(events) - 1
    if low < high:
        p = _partition(events, key, low, high)
        quick_sort(events, key, low, p - 1)
        quick_sort(events, key, p + 1, high)

def _partition(arr, key, low, high):
    pivot = key(arr[high]); i = low
    for j in range(low, high):
        if key(arr[j]) <= pivot:
            arr[i], arr[j] = arr[j], arr[i]; i += 1
    arr[i], arr[high] = arr[high], arr[i]
    return i


## 🧪 Simulação de Consumo de Insumos

In [6]:

ITEMS = ["Soro Fisiológico", "Luva Nitrílica", "Gaze Estéril", "Reagente X", "Seringa 5ml"]

def simulate_events(days=10, events_per_day=5, seed=42):
    random.seed(seed)
    start = datetime.now() - timedelta(days=days)
    events = []
    for d in range(days):
        day = start + timedelta(days=d)
        for _ in range(events_per_day):
            item = random.choice(ITEMS)
            qty = random.randint(1, 10)
            validity = day + timedelta(days=random.randint(30, 120))
            events.append(ConsumptionEvent(item=item, date=day, qty=qty, validity_until=validity))
    return events

events = simulate_events()
len(events), events[0]


(50,
 ConsumptionEvent(item='Soro Fisiológico', date=datetime.datetime(2025, 9, 6, 22, 32, 54, 925143), qty=1, validity_until=datetime.datetime(2025, 11, 10, 22, 32, 54, 925143)))

## 📑 Relatório Final

In [7]:

# Criar fila e pilha
q, s = Queue(), Stack()
for ev in events:
    q.enqueue(ev)
    s.push(ev)

# Ordenações
by_qty = merge_sort(events, key=lambda e: -e.qty)
by_validity = events[:]
quick_sort(by_validity, key=lambda e: e.validity_until)

# Buscas
indices = linear_search(events, lambda ev: ev.item == "Reagente X")
sorted_by_item_date = merge_sort(events, key=lambda e: (e.item, e.date))
pos_bin = -1
if indices:
    first_idx = indices[0]
    key = (events[first_idx].item, events[first_idx].date)
    pos_bin = binary_search(sorted_by_item_date, key)

# Relatório
print("="*60)
print("RELATÓRIO DE CONSUMO DE INSUMOS (Simulado)")
print("="*60)
print(f"Total de registros: {len(events)}")
print(f"Fila (FIFO): {len(q)} elementos")
print(f"Pilha (LIFO): {len(s)} elementos")

print("\nTop 3 consumos (Merge Sort):")
for ev in by_qty[:3]:
    print(f"  - {ev.item} | {ev.date.date()} | qtd {ev.qty} | validade {ev.validity_until.date()}")

print("\nPróximos a vencer (Quick Sort):")
for ev in by_validity[:3]:
    print(f"  - {ev.item} | validade {ev.validity_until.date()} | qtd {ev.qty}")

print("\nBusca sequencial (Reagente X):", len(indices), "ocorrências.")
print("Busca binária:", "encontrado na posição " + str(pos_bin) if pos_bin >= 0 else "não encontrado")

print("\nResumo:")
print(" - Fila (FIFO): registro cronológico.")
print(" - Pilha (LIFO): últimos eventos primeiro.")
print(" - Busca sequencial: simples, sem ordenação.")
print(" - Busca binária: exige ordenação, O(log n).")
print(" - Merge Sort: ordenação estável por quantidade.")
print(" - Quick Sort: ordenação in-place por validade.")


RELATÓRIO DE CONSUMO DE INSUMOS (Simulado)
Total de registros: 50
Fila (FIFO): 50 elementos
Pilha (LIFO): 50 elementos

Top 3 consumos (Merge Sort):
  - Reagente X | 2025-09-07 | qtd 10 | validade 2025-11-11
  - Gaze Estéril | 2025-09-09 | qtd 10 | validade 2025-11-24
  - Reagente X | 2025-09-13 | qtd 10 | validade 2025-12-03

Próximos a vencer (Quick Sort):
  - Seringa 5ml | validade 2025-10-10 | qtd 7
  - Seringa 5ml | validade 2025-10-13 | qtd 5
  - Gaze Estéril | validade 2025-10-15 | qtd 9

Busca sequencial (Reagente X): 8 ocorrências.
Busca binária: encontrado na posição 21

Resumo:
 - Fila (FIFO): registro cronológico.
 - Pilha (LIFO): últimos eventos primeiro.
 - Busca sequencial: simples, sem ordenação.
 - Busca binária: exige ordenação, O(log n).
 - Merge Sort: ordenação estável por quantidade.
 - Quick Sort: ordenação in-place por validade.


##**Explicação das Estruturas e Algoritmos Utilizados**

Para resolver o problema da falta de precisão no registro de consumo de insumos nas unidades de diagnóstico, foram aplicadas diferentes estruturas de dados e algoritmos clássicos, cada um com um papel específico:

1. Fila (Queue)
A fila foi utilizada para registrar o consumo diário de insumos na ordem cronológica em que ocorreram. Essa estrutura reflete o processo real de registro, onde os insumos consumidos em um determinado dia são adicionados ao final da fila e retirados na mesma ordem em que foram registrados (FIFO – First In, First Out). Isso permite acompanhar a evolução do consumo ao longo do tempo.

2. Pilha (Stack)
A pilha foi implementada para simular consultas em ordem inversa, ou seja, começando pelos insumos mais recentemente consumidos. Essa abordagem (LIFO – Last In, First Out) é útil em situações em que se deseja analisar rapidamente os últimos registros de consumo, por exemplo, para verificar um erro recente ou para auditoria imediata.

3. Busca Sequencial e Binária
Para localizar um insumo específico no registro, foram aplicadas duas estratégias:

- Busca Sequencial percorre toda a lista até encontrar o item desejado, sendo adequada para conjuntos de dados pequenos ou não ordenados.

- Busca Binária exige que os dados estejam previamente ordenados, mas permite encontrar rapidamente um insumo específico dividindo repetidamente a lista ao meio, reduzindo significativamente o tempo de busca em registros maiores.

4. Algoritmos de Ordenação (Merge Sort e Quick Sort)
Para organizar os insumos por quantidade consumida ou por data de validade, foram aplicados dois algoritmos de ordenação clássicos:

- Merge Sort, que divide os dados em sublistas menores e depois as combina ordenadas, garantindo eficiência mesmo em grandes conjuntos de dados.

- Quick Sort, que utiliza a técnica de partição em torno de um pivô, proporcionando rapidez na prática para organizar os registros.

Essas técnicas em conjunto permitem não apenas registrar e consultar o consumo diário de forma mais eficiente, mas também manter os dados organizados, possibilitando maior controle de estoque e previsão mais precisa da reposição de insumos.